In [3]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

In [4]:
# load the starts data
starts_df = pd.read_csv('~/capstone_data/skillshare_2022_starts_clean.csv')

In [5]:
# when saved as csv, the date columns tend to revert to str, so to be sure:
# make a day version of the trial start and end dates.
starts_df['trial_end_day'] = pd.to_datetime(starts_df.original_trial_end).dt.date
starts_df['trial_start_day'] = pd.to_datetime(starts_df.create_time).dt.date

In [6]:
# make a small df for merging onto the video views data.
trial_ends = starts_df[['user_uid', 'trial_start_day', 'trial_end_day']]

In [7]:
file_ends = [x for x in range(63)]

In [8]:
full_user_df = pd.DataFrame()

for x in [0, 1, 2]:
    print('working on file', x)
    filename = '~/capstone_data/skillshare_2022_vviews_'+str(x)+'.csv'
    vviews_df = pd.read_csv(filename)
    vviews_df['view_date_dt'] = pd.to_datetime(vviews_df.view_date).dt.date
    vviews_df = vviews_df.merge(trial_ends, how='left', left_on='uid', right_on='user_uid')
    trial_vviews_df = vviews_df[vviews_df['view_date_dt'] < vviews_df['trial_end_day']]
    trial_vviews_df['day_of_trial'] = (trial_vviews_df['view_date_dt'] - trial_vviews_df['trial_start_day']).dt.days + 1
    trial_vviews_df = trial_vviews_df[trial_vviews_df['day_of_trial'] > 0]
    ttv_df = trial_vviews_df.groupby(by=['uid', 'day_of_trial']).agg(minutes_watched=pd.NamedAgg(column='sum', aggfunc='sum')).reset_index()
    vviews_pivot = ttv_df.pivot_table(
        index=['uid'],
        columns='day_of_trial',
        values='minutes_watched').reset_index().rename_axis(None, axis=1).reset_index(drop=True).fillna(0.0)
    full_user_df = full_user_df.append(vviews_pivot, ignore_index=True)
    

working on file 0
working on file 1
working on file 2


In [9]:
for x in full_user_df.columns:
    if isinstance(x, int):
        colname = 'day-'+str(x)
        full_user_df.rename(columns={x: colname}, inplace=True)

In [10]:
print(len(full_user_df), len(full_user_df.uid.unique()))

15766 15766


In [12]:
full_user_df.to_csv('~/capstone_data/skillshare_2022_all_views.csv', index=False)